# Svenskt Kvinnobiografiskt lexikon part 1
version 1.3.5


* This [Jupyter Notebook](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon.ipynb) 
  * [part 2](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%202.ipynb) med sökfrågor etc.
  * [part 3](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%203.ipynb) hämtar data
* Video about this [swedish](https://www.youtube.com/watch?v=_LP48Yf6vgI) / [engelska](https://www.youtube.com/watch?v=wolyLu0WVsA)  
* Wikidata - [An introduction to Wikidata](https://www.youtube.com/watch?v=m_9_23jXPoE)
  * SKBL WIkidata property [P4963](https://www.wikidata.org/wiki/Property_talk:P4963) 
  

# Check diff SKBL Wikidata 
a try to synchWikidata / SKBL property P4963 
* [KARP APIet](https://spraakbanken.gu.se/verktyg/karp/webb-api)
  

In [1]:
from datetime import datetime
print("Last run: ", datetime.now())


Last run:  2020-09-15 23:20:37.800853


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None)
pd.set_option('display.max_rows', None)


dftot = pd.DataFrame()
 
url = "https://ws.spraakbanken.gu.se/ws/karp/v5/minientry?q=extended%7C%7Cand%7Cnamn%7Cexists&mode=skbllinks&show=name%2Curl&sort=sorteringsnamn.sort%2Csorteringsnamn.init%2Ctilltalsnamn.sort&size=10000&resource=skbl"
r = http.request('GET', url) 
data = json.loads(r.data.decode('utf-8')) 
#print(type(data))
#print(r.info())
listSKBL = []
i = 0 
for row in (data["hits"]["hits"]):
    #print (row["_source"]["url"])
    listSKBL.append(row["_source"]["url"])
    i = i + 1 
    #print (i)
SKBLtot = pd.DataFrame(listSKBL,
                  columns=['skblid'])


In [3]:
SKBLtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 1 columns):
skblid    1560 non-null object
dtypes: object(1)
memory usage: 12.3+ KB


In [4]:
#SKBLtot

In [5]:
#SKBLtot.sort_index(inplace=True) 
SKBLtot.sort_values(['skblid'], ascending=[1],inplace=True) 

In [6]:
# The API gives us some duplicates 
SKBLtot.drop_duplicates(subset ="skblid", keep = False, inplace = True)
SKBLtot.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1431 entries, 552 to 834
Data columns (total 1 columns):
skblid    1431 non-null object
dtypes: object(1)
memory usage: 22.4+ KB


In [7]:
SKBLtot = SKBLtot.reset_index(drop=True)  
#SKBLtot

# Wikidata

In [8]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT * WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P4963 ?skbl
} order by ?skbl"""


def get_results(endpoint_url, query):
    user_agent = "User:salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)
 
lst = []
for result in results["results"]["bindings"]:
    lst.append((result["skbl"]["value"]))
WDtot = pd.DataFrame(lst,columns={'skblid'}) 
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1494 entries, 0 to 1493
Data columns (total 1 columns):
skblid    1494 non-null object
dtypes: object(1)
memory usage: 11.8+ KB


In [9]:
WDtot.sort_index(inplace=True)  

In [10]:
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1494 entries, 0 to 1493
Data columns (total 1 columns):
skblid    1494 non-null object
dtypes: object(1)
memory usage: 11.8+ KB


In [11]:
#WDtot

### Check duplicates SKBL and Wikidata

In [12]:
dfmerge = pd.merge(WDtot, SKBLtot,how='outer', indicator=True)

In [13]:
# check whatsnew / deleted  
dfmerge['_merge'].value_counts()

both          1431
left_only       63
right_only       0
Name: _merge, dtype: int64

In [14]:
#dfmerge

In [15]:
SKBLnew = dfmerge[dfmerge['_merge']=="right_only"]


In [16]:
SKBLnew

,skblid,_merge


In [17]:
SKBLnew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 2 columns):
skblid    0 non-null object
_merge    0 non-null category
dtypes: category(1), object(1)
memory usage: 104.0+ bytes


In [18]:
end = datetime.now()
print("Ended: ", end)

Ended:  2020-09-15 23:20:39.644471
